In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# dataset = load_dataset("Amod/mental_health_counseling_conversations")
dataset = load_dataset("marmikpandya/mental-health")
dataset
# dataset['train'][0]

data.jsonl:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13358 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 13358
    })
})

In [ ]:
# model = AutoModelForCausalLM.from_pretrained('gpt2')

In [ ]:
#load tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
#gpt2 has no padding token by default. so end of sequence token is introduced
tokenizer.pad_token = tokenizer.eos_token

#tokenize the dataset
def tokenize_function(examples):
  #concatenate the context and response
  text = [f"{input} {output}" for input, output in zip(examples['input'], examples['output'])]
  inputs = tokenizer(text, truncation = True, padding='max_length', max_length = 512)
  inputs['labels'] = inputs['input_ids'].copy()
  return inputs

#apply the tokenize_function to each example in the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/13358 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13358
    })
})

Fine tuning the model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
model = AutoModelForCausalLM.from_pretrained('gpt2')

#training argument define
training_args = TrainingArguments(
    output_dir = './result',
    eval_strategy = 'epoch',
    num_train_epochs = 10,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    warmup_steps = 100,
    weight_decay = 0.01,
    logging_dir = './logs',
    report_to = 'none'
)

#initialize trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['train'].train_test_split(test_size = 0.2)
)

#train the model
trainer.train(resume_from_checkpoint='./result')
model_output_dir = './result/model'
model.save_pretrained(model_output_dir)
tokenizer.save_pretrained(model_output_dir)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Train Loss,Test Loss
1,0.527800,No log,0.448917,0.430761


('./result/model/tokenizer_config.json',
 './result/model/special_tokens_map.json',
 './result/model/vocab.json',
 './result/model/merges.txt',
 './result/model/added_tokens.json',
 './result/model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def get_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

def main():
    # Load the tokenizer and model from the saved directory
    model_path = './result/model'  # Adjust the path if necessary
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    # Calculate the Number of Parameters in the model being used for inference
    total_params = get_model_parameters(model)
    print(f"Total number of parameters: {total_params}")

    # Interactive input
    input_text = input("Give input.")

    # Prepare the input text you want to generate predictions for
    inputs = tokenizer(input_text, return_tensors='pt')

    # Generate Text
    outputs = model.generate(**inputs, max_length=50, num_return_sequences=1)

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("Generated Text:", generated_text)

# Call the main function
main()


In [ ]:
print("hello world")